<a href="https://colab.research.google.com/github/mmaronbr/alura-imersao-ia/blob/main/OrganizaMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependências da Aplicação

Dependências da Generative AI

In [ ]:
!pip install -U -q google.generativeai

Dependências do Gmail

In [ ]:
!pip install -U -q google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install -U -q pyngrok

# Importação das Bibliotecas

Bibliotecas de uso comum

In [ ]:
from google.colab import userdata

Bibliotecas utilizadas para acesso à API da Generative AI

In [ ]:
import google.generativeai as genai
import json
import random
import time

from google.api_core.exceptions import TooManyRequests


Bibliotecas utilizadas para acesso ao Gmail

In [ ]:
import os.path
import pickle
import requests

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from pyngrok import ngrok
from base64 import urlsafe_b64decode

# Configurações

## Generative AI

Definição da Chave da API da Generative AI

**IMPORTANTE!** Cadastre a secret `GOOGLE_API_KEY` na Seção de Secrets do Colab.

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Gmail

Configurações para acesso ao Gmail

**IMPORTANTE!** Cadastre as secrets: `CLIENT_ID`, `CLIENT_SECRET`,  `PROJECT_ID` e `NGROK_AUTHTOKEN` na Seção de Secrets do Colab.

Para obter o `CLIENT_ID`, `CLIENT_SECRET` e `PROJECT_ID` siga as orientações em: [Google Developers - Guia de Criação de Cliente OAuth](https://developers.google.com/workspace/guides/create-credentials?hl=pt-br#oauth-client-id)

Para o `NGROK_AUTHTOKEN`, registre-se em: [https://ngrok.com/](https://ngrok.com/)



Definição das constantes de autenticação/autorização

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/gmail.modify']

CLIENT_ID = userdata.get('CLIENT_ID')
CLIENT_SECRET = userdata.get('CLIENT_SECRET')
PROJECT_ID = userdata.get('PROJECT_ID')
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')

# Arquivo que armazena as configurações de acesso via OAuth2.0
CREDENTIALS_FILE = 'credentials.json'
# Arquivo que armazenará o token de acesso aos serviços da Google
TOKEN_PICKLE_FILE = 'token.pickle'
# Arquivo que armazena as configurações ngrok
NGROK_FILE = 'ngrok.yml'

# Configurações do Servidor Flow
PORT = 9999

Criação do arquivo de configuração do ngrok

In [ ]:
NGROK_TMP =f'''authtoken: {NGROK_AUTHTOKEN}
version: 2

tunnels:
  oauth_tunnel:
    proto: http
    addr: {PORT}
'''
# Cria o arquivo de Configurção do ngrok
with open(NGROK_FILE, "w") as n_file:
  n_file.write(NGROK_TMP)

Inicia o serviço ngrok em background

Para consultar os endpoints ativos, consulte: [Dashboard do Ngrok](https://dashboard.ngrok.com/cloud-edge/endpoints)

In [ ]:
#!nohup ngrok --config ngrok.yml start oauth_tunnel > nohup.out 2>&1 &
!nohup ngrok --config ngrok.yml > nohup.out 2>&1 &

Obtém o endereço público para comunicação com o Google

In [ ]:
ngrok.set_auth_token(NGROK_AUTHTOKEN)

public_url = ngrok.connect(PORT)
print(f'URL Público: {public_url.public_url}')
print('\nATENÇÃO! Cadastre o novo domínio do ngrok como um domínio válido para sua aplicação.\n')
print('Acesse: https://console.cloud.google.com/apis/credentials/consent')

configured = "não"

while configured.lower() != "sim":
  configured = input("Adicionou o endereço no ngrok como domínio autorizado?  ")

In [ ]:
JSON_TMP = f'''{{
    "installed": {{
      "client_id": "{CLIENT_ID}",
      "project_id": "{PROJECT_ID}",
      "auth_uri": "https://accounts.google.com/o/oauth2/auth",
      "token_uri": "https://oauth2.googleapis.com/token",
      "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
      "client_secret": "{CLIENT_SECRET}",
      "redirect_uris": ["{public_url.public_url}"]
    }}
  }}'''


# Cria o arquivo de Credenciais do Google
with open(CREDENTIALS_FILE, "w") as cred_file:
  cred_file.write(JSON_TMP)


# Definição das Funções

In [ ]:
def authenticate():
  creds = None

  # Recupera o token existente, caso o arquivo exista
  if os.path.exists(TOKEN_PICKLE_FILE):
    with open(TOKEN_PICKLE_FILE, 'rb') as token:
      creds = pickle.load(token)

  # Se as credenciais são inválidas, permite ao usuário se autenticar
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      # O Flow permite a conexão via OAuth2.0 para aplicações Desktop
      flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
      creds = flow.run_local_server(port=PORT, open_browser=False)

    # Save the credentials for the next run
    with open(TOKEN_PICKLE_FILE, 'wb') as token:
      pickle.dump(creds, token)

  return creds

In [ ]:
def list_inbox_messages():
  """Fetch emails from Gmail inbox."""
  creds = authenticate()
  service = build('gmail', 'v1', credentials=creds)

  # Call the Gmail API to fetch emails
  results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
  messages = results.get('messages', [])
  formatted_messages = []

  for message in messages[:10]:
    formatted_messages.append({'id':message['id'], 'content':get_message_content(message['id'])})

  return formatted_messages

In [ ]:
def get_message_content(message_id):
  """
  Retrieve sender, subject, snippet, and body content of a Gmail message.

  Args:
  message_id (str): The ID of the Gmail message.
  credentials: Google OAuth2 credentials object.

  Returns:
  dict: Dictionary containing sender, subject, snippet, and body content.
  """
  try:
    creds = authenticate()
    service = build('gmail', 'v1', credentials=creds)
    message = service.users().messages().get(userId='me', id=message_id, format='full').execute()

    # Extract sender
    sender = next(header['value'] for header in message['payload']['headers'] if header['name'] == 'From')

    # Extract subject
    subject = next(header['value'] for header in message['payload']['headers'] if header['name'] == 'Subject')

    # Extract snippet
    snippet = message['snippet']

    # Extract body content
    body_parts = []

    if 'data' in message['payload']['body']:
      body_data = message['payload']['body']['data']
      body_decoded = urlsafe_b64decode(body_data).decode('utf-8')
      body_parts.append(body_decoded)

    if 'parts' in message['payload']:
      # If the message contains parts, iterate through them to find the body
      for part in message['payload']['parts']:
        if part['mimeType'] == 'text/plain' or part['mimeType'] == 'text/html':
          body_data = part['body']['data']
          body_decoded = urlsafe_b64decode(body_data).decode('utf-8')
          body_parts.append(body_decoded)
        elif 'data' in part['body']:
          # If the part contains binary data (e.g., image), convert it to base64 and add to body parts
          part_data = part['body']['data']
          part_decoded = urlsafe_b64decode(part_data).decode('utf-8')
          body_parts.append(part_decoded)

    # Concatenate all body parts into a single string
    body = ''.join(body_parts)

    # Concatenate all attributes into a single string
    message_attributes = f'{sender}\n{subject}\n{snippet}\n{body}\n\n'
    return message_attributes

  except Exception as e:
    print("An error occurred:", e)
    return None

In [ ]:
def label_message(message_id, label_id):
  """Tag a label to a message in Gmail."""
  creds = authenticate()
  service = build('gmail', 'v1', credentials=creds)

  # Modify labels associated with the message
  service.users().messages().modify(userId='me', id=message_id, body={'addLabelIds': [label_id]}).execute()
  print(f'Marcador {label_id} adicionado a mensagem {message_id}.')

In [ ]:
def list_labels():
  """List all labels in Gmail"""
  creds = authenticate()
  service = build('gmail', 'v1', credentials=creds)

  service = build("gmail", "v1", credentials=authenticate())
  results = service.users().labels().list(userId="me").execute()
  labels = results.get("labels", [])

  return labels

In [ ]:
def create_label(label_name):
  """Create a new label in Gmail and return its ID."""
  creds = authenticate()
  service = build('gmail', 'v1', credentials=creds)

  # Create label request body
  label_body = {'name': label_name}

  # Call the Gmail API to create the label
  created_label = service.users().labels().create(userId='me', body=label_body).execute()

  label_id = created_label['id']
  print(f'Marcador {label_name} criado com o ID {label_id}.')

  return label_id

In [ ]:
def set_label_ids(categories):
    """Set the attribute label_id for each category based on label IDs obtained from Gmail."""
    labels = list_labels()
    for category in categories:
      for label in labels:
        if category['name'] == label['name']:
          category['label_id'] = label['id']
      if 'label_id' not in category:
        category['label_id'] = create_label(category['name'])
    return categories

In [ ]:
def get_random_number():
  """
  Generate a random number between 0.95 and 0.99.

  Returns:
  float: The random number.
  """
  return random.uniform(0.95, 0.99)

# Testes

Verifica se as configurações foram realizadas com sucesso para Generative AI

In [ ]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("Give me python code to sort a list")
print(response.text)

Verifica as configurações de autorização com o Google

In [ ]:
authenticate()

Verifica se a comunicação com a API do Gmail está funcional

Imprime os primeiros 5 marcadores (labels) do Gmail do usuário


In [ ]:
try:
    # Chama a API do Gmail
    service = build("gmail", "v1", credentials=authenticate())
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("Marcadores não encontrados")
    print("Labels:")
    for label in labels[:5]:
      print(f"[{label['id']}] {label['name']}")

except HttpError as error:
  print(f"An error occurred: {error}")

# OrganizaMail 💌

Definição das categorias, tipo de conteúdo esperado e textos de exemplo

In [ ]:
categories = [
    {
        'name': '[Follow-Up]',
        'description': 'Mensagens que demandam meu acompanhamento',
        'examples': ['informações importantes', 'aqui estão algumas atualizações', 'peço sua atenção', 'problemas de estabilidade']
    },
    {
        'name': '[Awaiting Response]',
        'description': 'Fios de mensagens que eu estou aguardando respostas.',
        'examples': ['Aguardo seu retorno']
    },
    {
        'name': '[To Read]',
        'description': 'Mensagens que podem ser lidas posteriormente.',
        'examples': ['Notícias', 'Novidades']
    },
]

Obtém os ID dos marcadores e associa com as categorias. Caso não existam, cria um novo marcador no Gmail

In [ ]:
categories = set_label_ids(categories)

Configurações iniciais do modelo generativo

In [ ]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Responda apenas no formato {'category':'<VALOR_DA_CATEGORIA>', 'probability':'<PROBABILIDADE>'}"

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)



Preparação do Histórico de acordo com a definição das Categorias

In [ ]:
num_categories = len(categories)
intro_part = f'Tenho {num_categories} categorias de classificação de mensagens: '

# Foi utilizado o label_id ao inves do name para facilitar o rotulo da msg
for index, category in enumerate(categories):
  intro_part += category['label_id']
  if index < num_categories-1:
    intro_part += ', '
  else:
    intro_part += '. Abaixo a descrição de cada uma delas:\n'

for category in categories:
  intro_part += f"{category['label_id']} - {category['description']}\n"

intro_part += "\nIdentifique as próximas mensagens de acordo com os exemplos:\n\n"

for category in categories:
  for example in category['examples']:
    intro_part += f"{example}\n'{{'category':'{category['name']}', 'probability':{get_random_number()}}}\n\n==="

intro_part
convo = model.start_chat()
convo.send_message(intro_part)
print(convo.last.text)

Categoriza as mensagens do Gmail

In [ ]:
messages = list_inbox_messages()
# Initialize the index of the last successful API call
last_successful_index = 0

for index, message in enumerate(messages[last_successful_index:], start=last_successful_index):
  try:
    convo.send_message(message['content'])
    response = json.loads(convo.last.text)
    label_message(message['id'], response['category'])
    # Update the index of the last successful API call
    last_successful_index = index
  except TooManyRequests as e:
    print("Limite de requisições excedido. Aguarde um momento...")
    time.sleep(20)
  except Exception as e:
    print("An error occurred:", e)
    break

# TODO: Para efeitos de teste, o OrganizaMail não está arquivando as mensagens ainda.